# DWD project - Team Anaconda

### file 3 - joining crime and housing data using SQL

### importing necesary packages for joining the tables

In [1]:
# using sqlite3 package to run SQL queries and pandas to fine-tune our final dataset
import sqlite3 as sq3
import pandas as pd

In [4]:
# opening database connection
connection = sq3.connect('Final_Database.db')

# creating the database cursor
conn_cur = connection.cursor() 

# Create table - crimes that corresponds to crime data csv file
conn_cur.execute('''CREATE TABLE crimes
             ([CMPLNT_NUM] VARCHAR,[ADDR_PCT_CD] VARCHAR, [KY_CD] VARCHAR, [OFNS_DESC] VARCHAR, [LAW_CAT_CD] VARCHAR, [BORO_NM] VARCHAR, [year_for_complaint] VARCHAR, [zipcode] VARCHAR)''')
          
# Create table - rental that corresponds to  housing data csv file
conn_cur.execute('''CREATE TABLE rental
             ([ZIP CODE] VARCHAR,[SALE YEAR] VARCHAR, [Median Sale Price] VARCHAR, [Sales Count] VARCHAR)''')
        

connection.commit()

### reading the csv files into the tables in the database

In [5]:
# reading the final_crime_file created from file 2
crime_data = pd.read_csv('C:/Users/aniru/Desktop/DWD/final_crime_file.csv')

# writing the crime_data dataframe to the crimes table in the database
crime_data.to_sql('crimes', connection, if_exists = 'replace', index = False)

# reading the Final_Median_Housing_File created from file 1
housing_data = pd.read_csv ('C:/Users/aniru/Desktop/DWD/Final_Median_Housing_Data.csv')

# writing the housing_data to the rental table in the database
housing_data.to_sql('rental', connection, if_exists = 'replace', index = False) 

C:\Anaconda\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


### Joining the tables 'rental' and 'crimes' that exist in the database

In [6]:
%%time
# the joining code takes around 45 minutes to run
conn_cur.execute('''
SELECT * 
from crimes
inner join rental on (crimes.zipcode || crimes.year_for_complaint = rental."ZIP CODE" || rental."SALE YEAR");
 ''')

# storing the joined table to an object called results
results = conn_cur.fetchall()

Wall time: 36min 41s


### Fine-tuning the final dataset

In [7]:
# extracting the column names and storing it in headers
headers = conn_cur.description

In [8]:
# assinging all column names in a list called hdr
hdr = []
for header in headers:
    hdr.append(header[0])

In [18]:
# assinging the joined dataframe to final_joined_df
final_joined_df = pd.DataFrame(results, columns = hdr)

In [19]:
# dropping columns that are not needed for further analysis
final_joined_df = final_joined_df.drop(final_joined_df.columns[[1, 8, 9]], axis=1)

In [21]:
# understanding what final_joined_df looks like
final_joined_df.head(5)

,CMPLNT_NUM,KY_CD,OFNS_DESC,LAW_CAT_CD,BORO_NM,year_for_complaint,zipcode,Median Sale Price,Sales Count
0,712500291,233,SEX CRIMES,MISDEMEANOR,BRONX,2010,10465,398265.5,204
1,919517331,235,DANGEROUS DRUGS,MISDEMEANOR,BRONX,2010,10456,377975.0,64
2,439147997,578,HARRASSMENT 2,VIOLATION,MANHATTAN,2010,10026,650000.0,247
3,697384902,360,None,MISDEMEANOR,BROOKLYN,2010,11218,495000.0,231
4,500286412,352,CRIMINAL TRESPASS,MISDEMEANOR,BROOKLYN,2010,11213,475000.0,127


In [16]:
# checking the number of rows in final_joined_df
len(final_joined_df)

3760638

In [24]:
# renaming the columns in final_joined_df
final_joined_df = final_joined_df.rename(columns = {'KY_CD':'OFNS_CODE','LAW_CAT_CD':'OFNS_LEVEL','BORO_NM':'BOROUGH','year_for_complaint':'YEAR','zipcode':'ZIPCODE','Median Sale Price':'MEDIAN_SALE','Sales Count':'SALES_COUNT'})

In [26]:
# closing the connection to the database
connection.close()

In [27]:
final_joined_df.to_csv('C:/Users/aniru/Desktop/DWD/final_joined_file.csv', index = False)